In [ ]:
import pickle

import numpy as np
import pymc3 as pm
import theano.tensor as tt
import pymbar

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from _BAR_estimator import get_values_from_trace
from _BAR_estimator import bayes_factor_smc
from _BAR_estimator import fit_normal_trace
from _BAR_estimator import draw_normal_samples
from _BAR_estimator import pot_ener_normal_aug
from _BAR_estimator import pot_ener
from _BAR_estimator import bootstrap_BAR
from _BAR_estimator import fit_uniform_trace
from _BAR_estimator import draw_uniform_samples
from _BAR_estimator import pot_ener_uniform_aug

In [ ]:
SEED = 2039
SIZE = 50

SMC_STEPS = 100000
MET_STEPS = 100000

# Data Generation

In [ ]:
def gaussian_mixture(mu1=0, sigma1=1, mu2=0, sigma2=1,
                     rho=0.5, size=100, random_state=None):
    rnd = np.random.RandomState(random_state)
    x = []
    for _ in range(size):
        if rnd.rand() < rho:
            x.append(rnd.normal(loc=mu1, scale=sigma1))
        else:
            x.append(rnd.normal(loc=mu2, scale=sigma2))
    return np.array(x)

In [ ]:
x = gaussian_mixture(mu1=0, sigma1=1, mu2=3, sigma2=1, rho=0.5, size=SIZE, random_state=SEED)
sns.distplot(x)

# Models

## Model 1

In [ ]:
with pm.Model() as model_m1:
    mu = pm.Uniform("mu", lower=-10, upper=10)
    sigma = pm.Uniform("sigma", lower=0.1, upper=5)
    obs = pm.Normal("obs", mu=mu, sigma=1, observed=x)

print(model_m1.vars)

## Model 2

In [ ]:
with pm.Model() as model_m2:
    w = pm.floatX([.5, .5])
    sigmas = pm.floatX([1., 1])
    
    mu = pm.Uniform("mu", lower=-10, upper=10)
    dmu = pm.Uniform("dmu", lower=0, upper=10)
    means = tt.zeros([2])
    means = tt.set_subtensor(means[0], mu)
    means = tt.set_subtensor(means[1], mu + dmu)
    
    sigma1 = pm.Uniform("sigma1", lower=0.1, upper=5)
    sigma2 = pm.Uniform("sigma2", lower=0.1, upper=5)
    sigmas = tt.zeros([2])
    sigmas = tt.set_subtensor(sigmas[0], sigma1)
    sigmas = tt.set_subtensor(sigmas[1], sigma2)
    
    obs = pm.NormalMixture("obs", w=w, mu=means, sigma=sigmas, observed=x)

print(model_m2.vars)

# Sequential Monte Carlo

In [ ]:
with model_m1:
    trace_m1 = pm.sample(SMC_STEPS, step=pm.SMC())
pm.traceplot(trace_m1[::10])

print("Log10 marginal likelihood: %0.5f" %(np.log10(model_m1.marginal_likelihood)))

In [ ]:
with model_m2:
    trace_m2 = pm.sample(SMC_STEPS, step=pm.SMC())
pm.traceplot(trace_m2[::10])

print("Log10 marginal likelihood: %0.5f" %(np.log10(model_m2.marginal_likelihood)))

In [ ]:
bf_smc_2_vs_1 = bayes_factor_smc(model_m1, model_m2)
print("Log10 of Bayes factor model m2 over model m1: %0.5f" % bf_smc_2_vs_1)

# BAR estimator

In [ ]:
def bayes_factor_uniform_aug(model_ini, sample_ini, model_fin, sample_fin, bootstrap=None):
    
    lower_upper_fin = fit_uniform_trace(sample_fin)
    lower_upper_fin = {"dmu_interval__": lower_upper_fin["dmu_interval__"],
                       "sigma2_interval__": lower_upper_fin["sigma2_interval__"]}
    
    # potential for sample drawn from i estimated at state i
    nsamples_ini = len(sample_ini["mu_interval__"])
    sample_aug_ini = draw_uniform_samples(lower_upper_fin, nsamples_ini)
    u_i_i = pot_ener_uniform_aug(sample_ini, model_ini, sample_aug_ini, lower_upper_fin)
    
    # potential for sample drawn from i estimated at state f
    sample_ini_comb = {"mu_interval__": sample_ini["mu_interval__"],
                       "sigma1_interval__": sample_ini["sigma_interval__"]}
    sample_ini_comb.update(sample_aug_ini)
    u_i_f = pot_ener(sample_ini_comb, model_fin)
    
    # potential for sample drawn from f estimated at state f
    u_f_f = pot_ener(sample_fin, model_fin)
    
    # potential for sample drawn from f estimated at state i
    sample_fin_split = {"mu_interval__": sample_fin["mu_interval__"], 
                        "sigma_interval__": sample_fin["sigma1_interval__"]}
    sample_aug_fin = {"dmu_interval__": sample_fin["dmu_interval__"], 
                      "sigma2_interval__": sample_fin["sigma2_interval__"]}
    u_f_i = pot_ener_uniform_aug(sample_fin_split, model_ini, sample_aug_fin, lower_upper_fin)
    
    w_F = u_i_f - u_i_i
    w_R = u_f_i - u_f_f

    delta_F = pymbar.BAR(w_F, w_R, compute_uncertainty=False, relative_tolerance=1e-12, verbose=True)
    bf = -delta_F

    if bootstrap is None:
        print("log10(bf) = %0.5f" % (bf *np.log10(np.e)))
        return bf
    else:
        print("Running %d bootstraps to estimate error." % bootstrap)
        bf_err = bootstrap_BAR(w_F, w_R, bootstrap)
        print("log10(bf) = %0.5f +/- %0.5f" % (bf * np.log10(np.e), bf_err * np.log10(np.e)))
        return bf, bf_err

In [ ]:
def repeat_sample(sample, repeats):
    sample = sample.copy()
    for k in sample.keys():
        sample[k] = np.repeat(sample[k], repeats)
    return sample

# Metropolis MC

In [ ]:
with model_m1:
    trace_m1 = pm.sample(MET_STEPS, step=pm.Metropolis())
pm.traceplot(trace_m1[::100])

In [ ]:
with model_m2:
    trace_m2 = pm.sample(MET_STEPS, step=pm.Metropolis())
pm.traceplot(trace_m2[::100])

In [ ]:
sample_m1 = get_values_from_trace(model_m1, trace_m1, thin=100, burn=100)
sample_m2 = get_values_from_trace(model_m2, trace_m2, thin=100, burn=100)

# Augmenting with uniformly-drawn samples

In [ ]:
bayes_factor_uniform_aug(model_m1, sample_m1, model_m2, sample_m2, bootstrap=100)

## Augmenting 10 more times

In [ ]:
sample_m1_repeat = repeat_sample(sample_m1, 10)
bayes_factor_uniform_aug(model_m1, sample_m1_repeat, model_m2, sample_m2, bootstrap=100)

## Augmenting 100 more times

In [ ]:
sample_m1_repeat = repeat_sample(sample_m1, 100)
bayes_factor_uniform_aug(model_m1, sample_m1_repeat, model_m2, sample_m2, bootstrap=100)